# Figure 9: Hydro projections

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd
import shapely.geometry

import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl     = px.colors.qualitative.D3
os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter("ignore")

# Ids for each glacier

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                        ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"] 
basins.loc[basins.basin_area > 5000, "Name"] = names # the space is important due to visualization purposes!

In [ ]:
# Glacier shapefiles
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 7][["RGIId", "Zone", "ID_basin"]]

RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 7]
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone', 'ID_basin' : 'ID_basin'})
RGI7_ids = RGI7_ids[["RGIId", "Zone", "ID_basin"]]

# merge both datasets
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")
dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

In [ ]:
# variables to analize
variables        = ['melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier']
scenarios        = ["ct_random", "ssp126","ssp245","ssp370","ssp585"]

def preprocessing(ds): # remove unnecessary variables and coordinates
    return ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])[variables]

def hydro_variables(ds): # calculate total_runoff, melt_on_glacier and smb
    ds["total_runoff"] = ((ds.melt_off_glacier + ds.melt_on_glacier + ds.liq_prcp_off_glacier + ds.liq_prcp_on_glacier)*1e-3)/(365*86400) # m3/s
    ds["melt_on_glacier"] = ((ds.melt_on_glacier)*1e-3)/(365*86400) # m3/s
    ds["ratio"] = ds["melt_on_glacier"] / ds["total_runoff"]
    return ds[variables]

def postprocessing(ds, scenario): # clean dataframe
    ds = ds.to_dataframe()
    ds["scenario"] = scenario
    ds = ds.set_index("scenario", append=True)
    ds = ds.reorder_levels(['scenario', 'rgi_id', 'time'])
    return ds

## Time series for each zone

In [ ]:
# historical period
all_combs = glob("/home/rooda/OGGM_results/new/*/run_outputs_*.nc", recursive = True)
all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocessing)

# assing zone to each glacier and aggregate the result 
ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
all_opts   = all_opts.groupby('rgi_id').sum()
all_opts   = all_opts.chunk("auto")

all_opts_total = all_opts.sum(dim = "rgi_id").assign_coords({"rgi_id": "total"}).expand_dims('rgi_id')
all_opts  = xr.concat([all_opts, all_opts_total], dim = "rgi_id")
all_opts  = hydro_variables(all_opts)
all_opts  = all_opts.isel(time = slice(0, -1))

# mean and standard desviation for each zone 
dataset_mean_hist = all_opts.mean(dim="options")
dataset_mean_hist = postprocessing(dataset_mean_hist, "historical")

dataset_var_hist = all_opts.std(dim="options")
dataset_var_hist = postprocessing(dataset_var_hist, "historical")

In [ ]:
dataset_mean = []
dataset_var  = []

for scenario in tqdm(scenarios): 
    all_combs = glob("/home/rooda/OGGM_results/new/*/run_output_*"+ scenario +"*.nc", recursive = True)
    all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocessing)

    if scenario == "ct_random":
        all_opts["time"] = all_opts["time"] + 2020
    
    # assing zone to each glacier and aggregate the result 
    ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
    all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
    all_opts   = all_opts.groupby('rgi_id').sum()
    all_opts_total = all_opts.sum(dim = "rgi_id").assign_coords({"rgi_id": "total"}).expand_dims('rgi_id')
    all_opts   = xr.concat([all_opts, all_opts_total], dim = "rgi_id")
    
    all_opts  = hydro_variables(all_opts)
    all_opts  = all_opts.chunk("auto")
    all_opts  = all_opts.isel(time = slice(0, -1))
    
    # standard desviation
    all_opts_var = all_opts.std(dim="options")
    all_opts_var = postprocessing(all_opts_var, scenario)
    dataset_var.append(all_opts_var)
    
    # mean
    all_opts_mean = all_opts.mean(dim="options")
    all_opts_mean = postprocessing(all_opts_mean, scenario)
    dataset_mean.append(all_opts_mean)
    
dataset_mean = pd.concat(dataset_mean)
dataset_var  = pd.concat(dataset_var)

dataset_var  = pd.concat([dataset_var_hist, dataset_var]).reset_index()
dataset_mean = pd.concat([dataset_mean_hist, dataset_mean]).reset_index()

In [ ]:
dict_replace = {"scenario": {"historical":'Historical', 
                             "ct_random":'Commitment run', 
                             "ssp126":'SSP 126', 
                             "ssp245":'SSP 245', 
                             "ssp370":'SSP 370', 
                             "ssp585":'SSP 585'}}

dataset_mean = dataset_mean.replace(dict_replace)
dataset_var  = dataset_var.replace(dict_replace)

## Peak water year for each catchment

In [ ]:
# historical period
all_combs = pd.read_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/dataset_hydro_signatures.csv", index_col = 0)
all_combs = all_combs.loc["peak_water_year"]
all_combs = all_combs.mean(axis = 0, skipna = True, numeric_only = True)
all_combs = all_combs.astype("int64")

all_combs = all_combs.rename("peak_water_year") 
all_combs.index = all_combs.index.astype("int64")

basins = pd.merge(basins, all_combs, left_index=True, right_index=True)

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

In [ ]:
scenarios   = ["Historical", "Commitment run", "SSP 126", "SSP 245", "SSP 370", "SSP 585"]
scen_colors = {"Historical":"rgba(0, 0, 0, 0.8)", "Commitment run":"rgba(0, 0, 0, 0.5)", "SSP 126":cl[0], "SSP 245":cl[2], "SSP 370":cl[1], "SSP 585":cl[3]}

zone1  = ['PPY', 'PCA']
zone2  = ['NPI-E','NPI-W']
zone3  = ['SPI-N', 'SPI-C', 'SPI-S']
zone4  = ['GCN', 'CDI']

fig    = make_subplots(rows=3, cols=3, horizontal_spacing = 0.04, vertical_spacing = 0.05, 
                       shared_xaxes= True, shared_yaxes= False, subplot_titles = ["Peak water ensemble mean", "Melt on glacier evolution (m<sup>3</sup> s<sup>-1</sup>)"],
                       specs=[[{"type": "scattergeo", "rowspan": 3},  {"type": "xy", "colspan": 2},           None],
                              [ None,                         {"type": "xy"},               {"type": "xy"}],
                              [ None,                         {"type": "xy"},               {"type": "xy"}]])

fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["rgba(213,213,213,0.6)", "rgba(213,213,213,0.6)"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

colorbar_volume = dict(len=0.45, x=0.2, y= 0.77, title='Year', thickness=20, title_font =  dict(size = 12))
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['peak_water_year'], 
                            colorscale = [(0.,"#fe7e0d"),(0.4, "#ffe9ba"),(1, "#1d78b4")], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 1980, zmax = 2080, colorbar = colorbar_volume), row=1, col=1)
fig.add_annotation(text="a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

## Add basin and hydrological zone names plus the hydro zone divides
fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.5, showlegend = False),row=1, col=1)  
fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.5)"), showlegend = False),row=1, col=1)
fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False,
                   textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)

for t in range(0,6): # for each scenario 
    
    # total volume
    time_series_id    = dataset_mean[dataset_mean.rgi_id == "total"][dataset_mean.scenario == scenarios[t]]
    time_series_sd_id = dataset_var[dataset_var.rgi_id == "total"][dataset_var.scenario == scenarios[t]]    
    
    fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.melt_on_glacier, mode='lines', name= scenarios[t], 
                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=True, legendgroup=t), row=1, col=2)
    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.melt_on_glacier+time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=1, col=2)
    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.melt_on_glacier-time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=1, col=2)
    fig.add_annotation(text="b) Total", font = dict(size = 13), x = 1985, y = 1100, showarrow=False, row=1, col=2)
    fig.update_yaxes(range = [1000,3000], dtick = 1000, row = 1, col =2)
                     
    # zone 1
    time_series_id    = dataset_mean[dataset_mean.rgi_id.isin(zone1)][dataset_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = dataset_var[dataset_var.rgi_id.isin(zone1)][dataset_var.scenario == scenarios[t]].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=time_series_id.melt_on_glacier, mode='lines', name= scenarios[t], 
                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=False, legendgroup=t), row=2, col=2)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier+time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=2, col=2)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier-time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=2, col=2)
    fig.add_annotation(text="c) PPY + PCA", font = dict(size = 13), x = 2000, y = 3, showarrow=False, row=2, col=2)
    fig.update_yaxes(range = [0,45], row = 2, col =2)
    
    # zone 2
    time_series_id    = dataset_mean[dataset_mean.rgi_id.isin(zone2)][dataset_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = dataset_var[dataset_var.rgi_id.isin(zone2)][dataset_var.scenario == scenarios[t]].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=time_series_id.melt_on_glacier, mode='lines', name= scenarios[t], 
                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=False, legendgroup=t), row=2, col=3)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier+time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=2, col=3)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier-time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=2, col=3)
    fig.add_annotation(text="d) NPI", font = dict(size = 13), x = 1990, y = 220, showarrow=False, row=2, col=3)
    fig.update_yaxes(range = [200,550], row = 2, col = 3)
    
    # zone 3
    time_series_id    = dataset_mean[dataset_mean.rgi_id.isin(zone3)][dataset_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = dataset_var[dataset_var.rgi_id.isin(zone3)][dataset_var.scenario == scenarios[t]].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=time_series_id.melt_on_glacier, mode='lines', name= scenarios[t], 
                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=False, legendgroup=t), row=3, col=2)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier+time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=3, col=2)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier-time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=3, col=2)
    fig.add_annotation(text="e) SPI", font = dict(size = 13), x = 1990, y = 600, showarrow=False, row=3, col=2)
    fig.update_yaxes(range = [500,2000], row = 3, col = 2)
    
    # zone 4
    time_series_id    = dataset_mean[dataset_mean.rgi_id.isin(zone4)][dataset_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = dataset_var[dataset_var.rgi_id.isin(zone4)][dataset_var.scenario == scenarios[t]].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=time_series_id.melt_on_glacier, mode='lines', name= scenarios[t], 
                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=False, legendgroup=t), row=3, col=3)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier+time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=3, col=3)
    fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=(time_series_id.melt_on_glacier-time_series_sd_id.melt_on_glacier), 
                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=3, col=3)
    fig.add_annotation(text="f) GCN + CDI", font = dict(size = 13), x = 2000, y = 60, showarrow=False, row=3, col=3)
    fig.update_yaxes(range = [50,200], row = 3, col = 3)
    
fig.update_layout(plot_bgcolor="rgba(213,213,213,0.6)")
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)    
    
fig.update_yaxes(tickangle = -90)
fig.update_xaxes(dtick = 20)
fig.update_layout(legend=dict(yanchor="top", y=1.0, orientation = "h", xanchor="left", x=0.35, bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(width = 1000, height = 600, margin = dict(l=20, r=20, b=20, t=20), hovermode = False)

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_9_hydro_projection.png", scale=4)
fig.show()